In [1]:
!export CUDA_VISIBLE_DEVICES=2

In [30]:
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler

In [3]:
with open('/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Text_Object/data/others/final_labels.pkl', 'rb') as file:
    final_labels = pickle.load(file)

In [63]:
objetos = ['OBJETO DE LA CONTRATACIÓN', 'OBJETO DE LA CONTRATACION', 'OBJETO DEI CONTRATO', 'OBJETO DEL CONTRATO', 'OBJECTE DE LA CONTRACTACIÓ', 'OBJETIVOS DEL CONTRATO', 'OBJETO DEL PROCEDIMIENTO DE CONTRATACIÓN', 'INFORMACIÓN SOBRE EL PROCEDIMIENTO DE CONTRATACIÓN']
objetos = [el.lower() for el in objetos]
objetos

['objeto de la contratación',
 'objeto de la contratacion',
 'objeto dei contrato',
 'objeto del contrato',
 'objecte de la contractació',
 'objetivos del contrato',
 'objeto del procedimiento de contratación',
 'información sobre el procedimiento de contratación']

In [4]:
df_zaragoza = pd.read_excel("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Text_Object/data/admin_eval_task/curated/tarea_zaragoza.xlsx")
df_zaragoza[df_zaragoza.objetivo != "/"]

,Unnamed: 0,procurement_id,doc_name,text,objetivo
0,128403,ntp00617381,ntp00617381_Pliego_Prescripciones_tecnicas_URI,pliego de prescripciones técnicas particulares...,dirección de ejecución de obra coordinacion de...
1,152306,ntp00552454,ntp00552454_Pliego_Prescripciones_tecnicas_URI,de ero en de aragon e ee instiuto aragonés l e...,el objeto del presente pliego es fijar las bas...
2,214074,ntp01175355,ntp01175355_Pliego_Prescripciones_tecnicas_URI,1ena aeropuerto de tenerife norte ciudad de la...,definir las condiciones por las que se ha de r...
3,156951,ntp00348490,ntp00348490_Pliego_Prescripciones_tecnicas_URI,segurtasun saila departamento de seguridad adm...,ejecución de la implantación de tomas de reca...
4,108923,ntp00573106,ntp00573106_Pliego_Prescripciones_tecnicas_URI,pliego de prescripciones técnicas para la cont...,el objeto del contrato es la completa ejecució...
...,...,...,...,...,...
68,196391,ntp00316053,ntp00316053_Pliego_Prescripciones_tecnicas_URI,sakidetza donostialdea esi pliego de bases téc...,el presente expediente de contratación tiene p...
69,232151,ntp01306212,ntp01306212_Pliego_Prescripciones_tecnicas_URI,liego de prescripciones técnicas para la contr...,prestación de un servicio de auditoría externa...
70,108219,ntp00496668,ntp00496668_Pliego_Prescripciones_tecnicas_URI,ip 5 expediente n2 pass 201911 universidad de ...,es objeto del presente pliego es establecer la...
71,80311,ntp01105326,ntp01105326_Pliego_Prescripciones_tecnicas_URI,_i 7 ul y un departamento de sanidad hospital ...,suministro de prótesis para el servicio de ang...


In [14]:
objetivos = df_zaragoza[df_zaragoza.objetivo != "/"].objetivo.values.tolist()
objetivos = [obj for obj in objetivos if not pd.isna(obj)]

In [45]:
# Load pre-trained SentenceTransformer model
model = SentenceTransformer("jaimevera1107/all-MiniLM-L6-v2-similarity-es")

positive_embeddings = model.encode(objetivos)

# Train One-Class SVM on the embeddings of positive questions
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
ocsvm.fit(positive_embeddings)

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OneClassSVM(gamma='auto', nu=0.1)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import OneClassSVM

vectorizer = CountVectorizer(max_features=5000)  

positive_embeddings = vectorizer.fit_transform(objetivos).toarray()

ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
ocsvm.fit(positive_embeddings)

OneClassSVM(gamma='auto', nu=0.1)

In [46]:
scores = ocsvm.decision_function(positive_embeddings)

scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(scores.reshape(-1, 1))

normalized_scores = normalized_scores.flatten()

In [8]:
df = pd.read_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_objectives/all_extracted_12aug_es/all_extracted_12aug_es_36_objectives.parquet")

In [9]:
test_pos = df[20:50]["objective"].iloc[1]
test_pos

'El objeto de este contrato es el Suministro, Instalación y Puesta en Funcionamiento de UN AUTOTRANSFUSOR con destino el Bloque Quirúrgico varios servicios del Complejo Asistencial Universitario de Salamanca.'

In [10]:
test = df[20:50]["objective"].iloc[3]
test

'El servicio se realizará conforme a lo establecido en el contrato, el Pliego de Cláusulas Administrativas Particulares y presente Pliego de Condiciones Técnicas. La duración del mismo será de 9 meses computándose desde el día 1 de octubre de 2018 o bien desde el día siguiente a la firma del contrato'

In [25]:
predictions = ocsvm.predict(vectorizer.transform([test_pos]).toarray())
predictions

array([-1])

In [49]:
new_data_embeddings = model.encode([test_pos, test])

new_scores = ocsvm.decision_function(new_data_embeddings)

normalized_new_scores = scaler.transform(new_scores.reshape(-1, 1))

normalized_new_scores = normalized_new_scores.flatten()

normalized_new_scores

array([0.35949105, 0.57887048])

In [53]:
df.columns

Index(['procurement_id_x', 'doc_name', 'translated', 'extracted',
       'clean_extracted', 'lang', 'objective', 'in_text_score', 'processed_0',
       'processed_1000', 'processed_2000', 'processed_3000', 'processed_4000'],
      dtype='object')

In [64]:
def is_in(x):
    x = x.lower()
    for el in objetos:
        if el in x:
            return el
    return "/"

In [65]:
df["is_in"] = df["clean_extracted"].apply(is_in)

In [60]:
len(df[df.is_in != "/"])

116

In [66]:
len(df[df.is_in != "/"])

6640

In [67]:
len(df)

10000

In [59]:
df[df.is_in != "/"][20:50].iloc[3]

procurement_id_x                                          ntp10513969
doc_name               ntp10513969_Pliego_Prescripciones_tecnicas_URI
translated                                                         no
extracted           PLIEGO DE PRESCRIPCIONES TÉCNICAS PARALA CONTR...
clean_extracted     PLIEGO DE PRESCRIPCIONES TÉCNICAS PARALA CONTR...
lang                                                         spa_Latn
objective           Contratación de los trabajos de adecuación de ...
in_text_score                                                     1.0
processed_0                                                      True
processed_1000                                                   True
processed_2000                                                   True
processed_3000                                                   True
processed_4000                                                   True
is_in                                       OBJETO DE LA CONTRATACIÓN
Name: 403604, dtype: